In [308]:
#Import Libraries

import sqlite3
import numpy as np 
import pandas as pd 
import pandasql as ps
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression

In [218]:
#Load data into DataFrames

df_sales = pd.read_csv('../Data/EXTR_RPSale.csv')
df_res = pd.read_csv('../Data/EXTR_ResBldg.csv')
df_lookup = pd.read_csv('../Data/EXTR_LookUp.csv')
df_parcel = pd.read_csv('../Data/EXTR_Parcel.csv', encoding='latin-1')

/Users/mihirbhagat/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (1,2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/mihirbhagat/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [310]:
#Create 'Year' and 'MajorMinor' columns in df_sales

df_sales['year'] = pd.DatetimeIndex(df_sales['DocumentDate']).year
df_sales['Major'] = df_sales['Major'].astype(str)
df_sales['Minor'] = df_sales['Minor'].astype(str)
df_sales['MajorMinor'] = df_sales['Major'] + '-' + df_sales['Minor']

In [311]:
#Filter for 2019 

df_sales_19 = df_sales[df_sales['year']==2019]

In [312]:
#Filter for PropertyType and SalePrice 

q1 = ("""SELECT * FROM df_sales_19
WHERE (PropertyType = 11 or PropertyType = 12 or PropertyType = 13 or PropertyType = 14) and SalePrice > 0
""")

In [313]:
df_sales_19 = ps.sqldf(q1)

In [314]:
#Create'MajorMinor' columns in df_res

df_res['Major'] = df_res['Major'].astype(str)
df_res['Minor'] = df_res['Minor'].astype(str)
df_res['MajorMinor'] = df_res['Major'] + '-' + df_res['Minor']

In [315]:
#Create'MajorMinor' columns in df_parcel

df_parcel['Major'] = df_parcel['Major'].astype(str)
df_parcel['Minor'] = df_parcel['Minor'].astype(str)
df_parcel['MajorMinor'] = df_parcel['Major'] + '-' + df_parcel['Minor']

In [316]:
#Merge Sales and Parcel 

new_df = pd.merge(df_sales_19, df_parcel, how = 'left',on =['MajorMinor'])

In [317]:
#Merge the merged DF with Res 

final_df = pd.merge(new_df, df_res, how = 'left',on =['MajorMinor'])

In [318]:
#Create a waterfront column 

final_df['is_waterfront'] = final_df['WfntLocation'] > 0

In [319]:
#How many waterfront in our DF 

final_df['WfntLocation'].value_counts()

0.0    19586
8.0       99
6.0       93
9.0       56
3.0       55
7.0       28
5.0        3
1.0        2
4.0        1
Name: WfntLocation, dtype: int64